In [3]:
from rbbm_src.labelling_func_src.src.example_tree_rules import *
from rbbm_src.labelling_func_src.src.LFRepair import run_snorkel


In [26]:
from rbbm_src.labelling_func_src.src.classes import clean_text
from snorkel.utils import probs_to_preds


In [5]:
import psycopg2
import pandas as pd
from snorkel.labeling import (
	LabelingFunction, 
	labeling_function, 
	PandasLFApplier, 
	LFAnalysis,
	filter_unlabeled_dataframe
	)

In [6]:
from snorkel.labeling.model import MajorityLabelVoter, LabelModel


In [7]:
conn = psycopg2.connect(dbname='label', user='postgres')

In [8]:
# amazon_rules = gen_amazon_funcs()
imdb_rules=gen_imdb_funcs()

In [9]:
# lfs=[f.gen_label_rule() for f in amazon_rules]
lfs = [f.gen_label_rule() for f in imdb_rules]

In [10]:
# logger.critical(LFs)
sentences_df=pd.read_sql(f'SELECT * FROM amazon01', conn)
sentences_df = sentences_df.rename(columns={"class": "expected_label", "content": "old_text"})
sentences_df['text'] = sentences_df['old_text'].apply(lambda s: clean_text(s))
sentences_df = sentences_df[~sentences_df['text'].isna()]

# lf_internal_args = lf_input_internal(funcs=LFs)

# # Snorkel built-in labelling function applier
applier = PandasLFApplier(lfs=lfs)

# # Apply the labelling functions to get vectors
initial_vectors = applier.apply(df=sentences_df, progress_bar=False)
# # logger.critical(initial_vectors)
# # df.to_csv('snorkel.csv')

# if(lf_input.training_model_type=='majority'):
#     # pdb.set_trace()
#     model = MajorityLabelVoter()
# else:
model = LabelModel(cardinality=2, verbose=True, device='cpu')
model.fit(L_train=initial_vectors, n_epochs=500, log_freq=100, seed=123)
# snorkel needs to get an estimator using fit function first
# training model with all labelling functions

probs_test= model.predict_proba(L=initial_vectors)

# df_sentences_filtered, probs_test_filtered = filter_unlabeled_dataframe(
#         X=sentences_df, y=probs_test, L=initial_vectors
# )
# # reset df_train to those receive signals
# df_sentences_filtered = df_sentences_filtered.reset_index(drop=True)
# # df_sentences_filtered=sentences_df
# filtered_vectors=initial_vectors
# filtered_vectors = applier.apply(df=df_sentences_filtered, progress_bar=False)
# cached_vectors = dict(zip(LFs, np.transpose(filtered_vectors)))
# lf_internal_args.func_vectors = cached_vectors

# # logger.critical(cached_vectors)
# if(lf_input.training_model_type=='snorkel'):
#     model.fit(L_train=filtered_vectors, n_epochs=500, log_freq=100, seed=123)

# df_sentences_filtered['model_pred'] = pd.Series(model.predict(L=filtered_vectors))

# df_sentences_filtered['vectors'] = pd.Series([",".join(map(str, t)) for t in filtered_vectors])

# lf_internal_args.filtered_sentences_df = df_sentences_filtered

# df_sentences_filtered.to_csv('result.csv')
# # the wrong labels we get
# wrong_preds = df_sentences_filtered[(df_sentences_filtered['expected_label']!=df_sentences_filtered['model_pred'])]
# # df_sentences_filtered.to_csv('predictions_shakira.csv', index=False)
# wrong_preds['signal_strength'] = wrong_preds['vectors'].apply(lambda s: sum([1 for i in s.split(",") if int(i) == SPAM or int(i)==HAM]))
# wrong_preds = wrong_preds.sort_values(['signal_strength'], ascending=False)
# # logger.critical(wrong_preds)
# accuracy=(len(df_sentences_filtered)-len(wrong_preds))/len(df_sentences_filtered)
# logger.critical(f"""
#     out of {len(sentences_df)} sentences, {len(df_sentences_filtered)} actually got at least one signal to \n
#     make prediction. Out of all the valid predictions, we have {len(wrong_preds)} wrong predictions, \n
#     accuracy = {(len(df_sentences_filtered)-len(wrong_preds))/len(df_sentences_filtered)} 
#     """)

# return accuracy, df_sentences_filtered, wrong_preds

/tmp/ipykernel_12733/3448868205.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sentences_df=pd.read_sql(f'SELECT * FROM imdb', conn)
INFO 2024-05-07 17:40:59,500 [label_model.py:fit:905] Computing O...
INFO 2024-05-07 17:40:59,508 [label_model.py:fit:911] Estimating \mu...
  0%|                                                                                                                                                                                                                                                | 0/500 [00:00<?, ?epoch/s]INFO 2024-05-07 17:40:59,514 [logger.py:log:79] [0 epochs]: TRAIN:[loss=0.039]
INFO 2024-05-07 17:40:59,585 [logger.py:log:79] [100 epochs]: TRAIN:[loss=0.001]
 56%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [38]:
df_sentences_filtered, probs_test_filtered = filter_unlabeled_dataframe(
        X=sentences_df, y=probs_test, L=initial_vectors
)

In [39]:
df_sentences_filtered.shape

(24161, 5)

In [40]:
filtered_vectors = applier.apply(df=df_sentences_filtered, progress_bar=False)

In [41]:
filtered_vectors.shape

(24161, 7)

In [43]:
import numpy as np

In [44]:
has_nan = np.isnan(filtered_vectors).any()


In [48]:
df_sentences_filtered = df_sentences_filtered.reset_index()

In [49]:
df_sentences_filtered['vectors'] =  pd.Series([",".join(map(str, t)) for t in filtered_vectors])

In [50]:
df_sentences_filtered['vectors']

0         -1,-1,-1,1,1,-1,-1
1        -1,0,-1,-1,-1,-1,-1
2        0,-1,-1,-1,-1,-1,-1
3        -1,0,-1,-1,-1,-1,-1
4        -1,-1,-1,-1,-1,0,-1
                ...         
24156     -1,0,-1,1,-1,-1,-1
24157    -1,-1,-1,1,-1,-1,-1
24158     -1,-1,-1,1,1,-1,-1
24159    -1,-1,-1,-1,1,-1,-1
24160     0,-1,-1,1,-1,-1,-1
Name: vectors, Length: 24161, dtype: object

In [51]:
probs_test_filtered

array([[1.65185586e-01, 8.34814414e-01],
       [9.99978342e-01, 2.16580380e-05],
       [9.99965747e-01, 3.42530487e-05],
       ...,
       [1.65185586e-01, 8.34814414e-01],
       [3.29956851e-01, 6.70043149e-01],
       [9.99914759e-01, 8.52411414e-05]])

In [59]:
prob_diffs_tuples = [(t[0],t[1]) for t in probs_test_filtered]
df_sentences_filtered['model_pred_prob_tuple'] = pd.Series(prob_diffs_tuples)
prob_diffs = [abs(t[0]-t[1]) for t in probs_test_filtered]
df_sentences_filtered['model_pred_diff'] = pd.Series(prob_diffs)

In [53]:
probs_test_filtered.shape

(24161, 2)

In [54]:
df_sentences_filtered.shape

(24161, 8)

In [55]:
df_sentences_filtered['pred'] = probs_to_preds(probs=probs_test_filtered)

In [62]:
df_sentences_filtered['model_pred'] = pd.Series(model.predict(L=filtered_vectors))

In [63]:
df_sentences_filtered.head()

,index,old_text,expected_label,fold,cid,text,vectors,model_pred_prob_tuple,pred,model_pred_diff,model_pred
0,7,my fondness for chris rock varies with his moviesi hated him after lethal weapon 4but i hated everyone in that movie after iti like him when he is himself and not holding backlike in dogma well this is his best yetwasnt expecting this to be that goodlaughed my arse off the whole time chris rock delivers a sweet wonderful story backed by some of the funniest comedy ive seen in quite some time loved it,1,0,8,my fondness for chris rock varies with his moviesi hated him after lethal weapon but i hated everyone in that movie after iti like him when he is himself and not holding backlike in dogma well this is his best yetwasnt expecting this to be that goodlaughed my arse off the whole time chris rock delivers a sweet wonderful story backed by some of the funniest comedy ive seen in quite some time loved it,"-1,-1,-1,1,1,-1,-1","(0.1651855858256744, 0.8348144141743256)",1,0.669629,1
1,8,tony arzenta a sicilian hit man or professional killer decides to leave the business and his former employers do not agreein terms of content this highly enjoyable action movie doesnt have one in terms of sheer amusement it is funit is very melodramatic violent quite brutal the car chases are notable arzenta is an unpretentious yet very likable filmmuch better than the current hollywoodian trash that gets the same label it comes from delons rather short flirting with the italian b cinema of the 70s it carefully uses delons tough guy persona belonging to the gallery of bad ass thugs that he made in his youth i enjoyed very much the fact that delon made this film that he had a role in a good eurocrime flick the score is very fine with a good introductory songmaking felt that gusto that the italians had for the film considered as a synthetic workwhere the musical art has an important part in tony arzenta big guns no way out the very appealing erika blanc 31 years in 73 appears as an unnamed hookermeanwhile arzentas girlfriend sandrais played by carla gravina a starlet that practically left the movies after 75 arzenta is interestingly filmedan ambitious visual conception some expressionistic peculiar angles the movie was directed by the prolific duccio tessarithe one who made also zorro the delon comedyneedless to mention that these two filmstony arzenta 1973 and zorro are very unlikethe first one is a bloody melodramatic violent action movie the second is a lighthearted comedymore kindred to a spoofthough remarkably coherent and skilfully madeduccio tessari directed films like kiss kiss bang bang 1966sons of satan the bloodstained butterfly ¡viva la muerte tua tough guys 1974safari express 1976 etc,1,0,9,tony arzenta a sicilian hit man or professional killer decides to leave the business and his former employers do not agreein terms of content this highly enjoyable action movie doesnt have one in terms of sheer amusement it is funit is very melodramatic violent quite brutal the car chases are notable arzenta is an unpretentious yet very likable filmmuch better than the current hollywoodian trash that gets the same label it comes from delons rather short flirting with the italian b cinema of the s it carefully uses delons tough guy persona belonging to the gallery of bad ass thugs that he made in his youth i enjoyed very much the fact that delon made this film that he had a role in a good eurocrime flick the score is very fine with a good introductory songmaking felt that gusto that the italians had for the film considered as a synthetic workwhere the musical art has an important part in tony arzenta big guns no way out the very appealing erika blanc years in appears as an unnamed hookermeanwhile arzentas girlfriend sandrais played by carla gravina a starlet that practically left the movies after arzenta is interestingly filmedan ambitious visual conception some expressionistic peculiar angles the movie was directed by the prolific duccio te

In [61]:
df_sentences_filtered.sort_values(by='model_pred_diff')

,index,old_text,expected_label,fold,cid,text,vectors,model_pred_prob_tuple,pred,model_pred_diff
1639,4215,sam fullers excellent pick up on south street is the pick of the bunch from a number of early 50s cold war influenced low budget noir vehicles with a running length of under 80 minutes pick up on south street is tough gritty explosive and endlessly entertaining widmark stars as pickpocket skip mccoy who has already been picked up three times yet mccoy cant keep his wandering fingers out of trouble and trouble is exactly what he slides into when he grifts the wallet of gangsters moll candy jean peters candys wallet contains a roll of microfilm invaluable to the communist movement and its her last job for ex boyfriend richard kiley to make the delivery however when widmark lifts it peters must do whatever it takes to re claim the film she initially knows nothing about its a tasty set up with widmarks character while not the psycho of kiss of death a real live wire unpredictable and tough yet curiously charmingwhen bogart or mitchum stepped into a film noir role you knew what you were going to get a lone anti hero maintaining his moral integrity and winning out in the end bogart or an overly laconic guy who allows himself to be drawn into a trap mitchum with widmark you just dont know what you are going to get and with his incredibly modern acting style his films always hold up well he is amazing to watch here he is torn between making a big score for himself by selling the film or handing it over to the police and fighting the commies on the right side of the law and he still has to pretend he never pickpocketed peters to avoid the fatal fourth rap on his sheet peters gets her best role as the moll with a heart of gold candy widmarks unpredictability is perhaps best expressed in his scenes with peters the gorgeous tramp quickly and rather unbelievably the romance angle is rather rushedfalls under widmarks spell yet widmark alternates between kissing her or slapping her around peters hard edged beauty yet lack of over lacquered hollywood glamour lana turner would never have worked well in this role is a major asset to the film candy is not innocent yet shes very vulnerable constantly being passed between and slapped around by men widmark knocks her cold on first meeting and wakes her by pouring beer over her face yet by the final act hes a lot more tender to her after she cops one hell of a going over from kiley the scene in the hospital with peters and widmark shouldnt work but it does thelma ritter is brilliant as stoolie moe well deserving of her oscar nomination ritters performance like everything else in the film is gritty real and heartbreakingly honest her death scene is stunning fullers camera movements and location settings are particularly interesting fuller loved a good close up and pick up on south street is full of uncomfortable cloistering tight shots that only enhance the tension of the plot fuller isnt afraid to let the camera linger on a shot for longer than standard old hollywood really allowed yet stunningly pulls away from ritters death scene to give the audience maximum impact the urban locales and unusual confronting camera angles give pick up on south street a bold uncompromisingly modern look 10 10,1,0,4216,sam fullers excellent pick up on south street is the pick of the bunch from a number of early s cold war influenced low budget noir vehicles with a running length of under minutes pick up on south street is tough gritty explosive and endlessly entertaining widmark stars as pickpocket skip mccoy who has already been picked up three times yet mccoy cant keep his wandering fingers out of trouble and trouble is exactly what he slides into when he grifts the wallet of gangsters moll candy jean peters candys wallet contains a roll of microfilm invaluable to the communist movement and its her last job for ex boyfriend richard kiley to make the delivery however when widmark lifts it peters must do whatever 

In [15]:
sentences_df['prob_diff'] = pd.Series(prob_diffs)

In [16]:
sentences_df.shape

(19943, 6)

In [73]:
def stats_by_confidence_diff(threshold, inputs):
    g_thresh = [t for t in inputs if abs(t[1][0]-t[1][1])>threshold]
    le_thresh = [t for t in inputs if abs(t[1][0]-t[1][1])<=threshold]
    
    correct_preds_from_g_thresh = [t for t in g_thresh if t[0]==t[2]]
    correct_preds_from_le_thresh = [t for t in le_thresh if t[0]==t[2]]
    
    print(f" #instances > {round(threshold,1)} prob diff: {len(g_thresh)}")
    print(f" #instances <= {round(threshold,1)} prob diff: {len(le_thresh)}")
    print(f"accuracy above {round(threshold,1)} prob diff: {len(correct_preds_from_g_thresh)/len(g_thresh)}")
    print(f"accuracy below/equal {round(threshold,1)} prob diff: {len(correct_preds_from_le_thresh)/len(le_thresh)}")
    
    return g_thresh, le_thresh

In [58]:
stats_by_confidence_diff(0.1, triples)

 #instances > 0.1 prob diff: 18251
 #instances <= 0.1 prob diff: 1692
accuracy above 0.1 prob diff: 0.6423209687140431
accuracy below/equal 0.1 prob diff: 0.031914893617021274


([(1, array([0.27321541, 0.72678459]), 1),
  (1, array([0.04708405, 0.95291595]), 1),
  (1, array([0.80079124, 0.19920876]), 0),
  (0, array([0.27321216, 0.72678784]), 1),
  (0, array([9.99910323e-01, 8.96774236e-05]), 0),
  (1, array([0.28945918, 0.71054082]), 1),
  (1, array([1.09722573e-06, 9.99998903e-01]), 1),
  (1, array([0.91448216, 0.08551784]), 0),
  (0, array([0.05901095, 0.94098905]), 1),
  (1, array([0.04098879, 0.95901121]), 1),
  (0, array([0.57359566, 0.42640434]), 0),
  (1, array([0.05436768, 0.94563232]), 1),
  (0, array([0.57359566, 0.42640434]), 0),
  (1, array([0.27321216, 0.72678784]), 1),
  (0, array([9.99992087e-01, 7.91268644e-06]), 0),
  (1, array([0.01375585, 0.98624415]), 1),
  (0, array([0.09506586, 0.90493414]), 1),
  (1, array([0.27321216, 0.72678784]), 1),
  (0, array([9.99997025e-01, 2.97453086e-06]), 0),
  (1, array([0.27321216, 0.72678784]), 1),
  (1, array([0.01180302, 0.98819698]), 1),
  (1, array([0.04098814, 0.95901186]), 1),
  (1, array([0.3312329

In [51]:
import numpy as np

In [52]:
for i in np.arange(0, 1, 0.1):
    stats_by_confidence_diff(i, triples)
    print('\n')

 #instances > 0.0 prob diff: 18506
 #instances <= 0.0 prob diff: 1437
accuracy above 0.0 prob diff: 0.6363881984221333
accuracy below/equal 0.0 prob diff: 0.0


 #instances > 0.1 prob diff: 18251
 #instances <= 0.1 prob diff: 1692
accuracy above 0.1 prob diff: 0.6423209687140431
accuracy below/equal 0.1 prob diff: 0.031914893617021274


 #instances > 0.2 prob diff: 16837
 #instances <= 0.2 prob diff: 3106
accuracy above 0.2 prob diff: 0.6342578844212152
accuracy below/equal 0.2 prob diff: 0.3535093367675467


 #instances > 0.3 prob diff: 16183
 #instances <= 0.3 prob diff: 3760
accuracy above 0.3 prob diff: 0.6424643144040042
accuracy below/equal 0.3 prob diff: 0.3670212765957447


 #instances > 0.4 prob diff: 14970
 #instances <= 0.4 prob diff: 4973
accuracy above 0.4 prob diff: 0.6699398797595191
accuracy below/equal 0.4 prob diff: 0.3514980896842952


 #instances > 0.5 prob diff: 12269
 #instances <= 0.5 prob diff: 7674
accuracy above 0.5 prob diff: 0.6837558073192599
accuracy below

In [74]:
# accuracy below/equal 0.1 
g_thresh01, le_thresh01 = stats_by_confidence_diff(0.1, inputs)

# correct_preds_from_g_thresh = [t for t in g_thresh if t[0]==t[2]]
# correct_preds_from_le_thresh = [t for t in le_thresh if t[0]==t[2]]
    

 #instances > 0.1 prob diff: 18251
 #instances <= 0.1 prob diff: 1692
accuracy above 0.1 prob diff: 0.6423209687140431
accuracy below/equal 0.1 prob diff: 0.031914893617021274


In [76]:
for t in le_thresh01:
    print(t)
#     break

(1, array([0.5, 0.5]), -1, 'series has gone on too long she needs to end this seriesthe last few books have been babblingnot worth the time it takes to read a few pages')
(0, array([0.5, 0.5]), -1, 'so far so good waiting to see if strain will straighten out  using it to gather underbrush with a winch  so far so good  waiting to see if strain will straighten out the tinesso  far it has not')
(0, array([0.4849522, 0.5150478]), 1, 'cool design awesome design  i have a set on my dining table that i love  would recommend')
(1, array([0.5, 0.5]), -1, 'there are many better books out there i had a hard time reading this book ironically enough due to the way it was organized it would have been much more helpful if the book had been broken up by lifestyles rather than by area of organization i had to keep jumping around between chapter to read the information that was important to me')
(0, array([0.5, 0.5]), -1, 'fun adventure this is a lot of fun  although you suspect the ex is doing somethin

In [ ]:
f1 = keyword_labelling_func_builder(['star', 'stars'], HAM)
f2 = keyword_labelling_func_builder(['product', 'fit', 'quality', 'size', 'cheap', 'wear'], SPAM)
f3 = keyword_labelling_func_builder(['great'], HAM)

# f4 is an estension of f3
cur_number=1
tree_size=1
f4_root = PredicateNode(number=cur_number, pred=KeywordPredicate(keywords=['great']))
cur_number+=1
tree_size+=1
f4_root_left_leaf = LabelNode(number=cur_number, label=ABSTAIN, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
cur_number+=1
tree_size+=1
f4_root_right_child = PredicateNode(number=cur_number, pred=KeywordPredicate(keywords=['stars','works']))
cur_number+=1
tree_size+=1
f4_root_right_child_left_leaf = LabelNode(number=cur_number, label=ABSTAIN, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
cur_number+=1
tree_size+=1
f4_root_right_child_right_leaf = LabelNode(number=cur_number, label=HAM, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
f4_root.left = f4_root_left_leaf
f4_root_left_leaf.parent=f4_root
f4_root.right = f4_root_right_child
f4_root_right_child.parent=f4_root
f4_root_right_child.left = f4_root_right_child_left_leaf
f4_root_right_child_left_leaf.parent=f4_root_right_child
f4_root_right_child.right = f4_root_right_child_right_leaf
f4_root_right_child_right_leaf.parent=f4_root_right_child
f4 = TreeRule(rtype='lf', root=f4_root, size=tree_size, max_node_id=cur_number, is_good=True)

# f4 = keyword_labelling_func_builder(['great','stars','works'], HAM)
f5 = keyword_labelling_func_builder(['waste'], SPAM)
f6 = keyword_labelling_func_builder(['shoes','item','price','comfortable','plastic'], HAM)
f7 = keyword_labelling_func_builder(['junk','bought','like','dont','just','use','buy','work','small','didnt','did','disappointed'], SPAM)

# f8 is an estension of f7
cur_number=1
tree_size=1
f8_root = PredicateNode(pred=KeywordPredicate(keywords=['junk','bought','like','dont','just','use','buy','work','small','didnt','did','disappointed']))
cur_number+=1
tree_size+=1
f8_root_left_leaf = LabelNode(number=cur_number, label=ABSTAIN, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
cur_number+=1
tree_size+=1
f8_root_right_child = PredicateNode(number=cur_number, pred=KeywordPredicate(keywords=['shoes','metal','fabric','replace','battery','warranty','plug']))
cur_number+=1
tree_size+=1
f8_root_right_child_left_leaf = LabelNode(number=cur_number, label=ABSTAIN, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
cur_number+=1
tree_size+=1
f8_root_right_child_right_leaf = LabelNode(number=cur_number, label=SPAM, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))

f8_root.left = f8_root_left_leaf
f8_root_left_leaf.parent=f8_root

f8_root.right = f8_root_right_child
f8_root_right_child.parent=f8_root

f8_root_right_child.left = f8_root_right_child_left_leaf
f8_root_right_child_left_leaf.parent=f8_root_right_child
f8_root_right_child.right = f8_root_right_child_right_leaf
f8_root_right_child_right_leaf.parent=f8_root_right_child
f8 = TreeRule(rtype='lf', root=f8_root, size=tree_size, max_node_id=cur_number, is_good=True)


# f8 = keyword_labelling_func_builder(['junk','bought','like','dont','just','use','buy','work','small','didnt','did','disappointed',
# 	'shoes','metal','fabric','replace','battery','warranty','plug'], SPAM)

f9  = keyword_labelling_func_builder(['love', 'perfect', 'loved', 'nice', 'excellent', 'works', 'loves', 'awesome', 'easy'], HAM)

# f10 is an estension of f9
cur_number=1
tree_size=1
f10_root = PredicateNode(number=cur_number, pred=KeywordPredicate(keywords=['love', 'perfect', 'loved', 'nice', 'excellent', 'works', 'loves', 'awesome', 'easy']))
cur_number+=1
tree_size+=1
f10_root_left_leaf = LabelNode(number=cur_number, label=ABSTAIN, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
cur_number+=1
tree_size+=1
f10_root_right_child = PredicateNode(number=cur_number, pred=KeywordPredicate(keywords=['stars', 'soft']))
cur_number+=1
tree_size+=1
f10_root_right_child_left_leaf = LabelNode(number=cur_number, label=ABSTAIN,  pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
cur_number+=1
tree_size+=1
f10_root_right_child_right_leaf = LabelNode(number=cur_number, label=HAM,  pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
f10_root.left = f10_root_left_leaf
f10_root_left_leaf.parent=f10_root
f10_root.right = f10_root_right_child
f10_root_right_child.parent=f10_root
f10_root_right_child.left = f10_root_right_child_left_leaf
f10_root_right_child_left_leaf.parent=f10_root_right_child
f10_root_right_child.right = f10_root_right_child_right_leaf
f10_root_right_child_right_leaf.parent=f10_root_right_child
f10 = TreeRule(rtype='lf', root=f10_root,size=tree_size, max_node_id=cur_number, is_good=True)
# f10  = keyword_labelling_func_builder(['love', 'perfect', 'loved', 'nice', 'excellent', 'works', 'loves', 'awesome', 'easy', 'stars', 'soft'], HAM)


# f11 is an estension of f9
cur_number=1
tree_size=1
f11_root = PredicateNode(number=cur_number,pred=KeywordPredicate(keywords=['love', 'perfect', 'loved', 'nice', 'excellent', 'works', 'loves', 'awesome', 'easy']))
cur_number+=1
tree_size+=1
f11_root_left_leaf = LabelNode(number=cur_number,label=ABSTAIN, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
cur_number+=1
tree_size+=1
f11_root_right_child = PredicateNode(number=cur_number,pred=KeywordPredicate(keywords=['shoes', 'bought', 'use', 'purchase', 'purchased', 'colors', 'install', 'clean', 'design',
 'pair', 'screen', 'comfortable', 'products']))
cur_number+=1
tree_size+=1
f11_root_right_child_left_leaf = LabelNode(number=cur_number,label=ABSTAIN, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
cur_number+=1
tree_size+=1
f11_root_right_child_right_leaf = LabelNode(number=cur_number,label=HAM, pairs={SPAM:[], HAM:[]}, used_predicates=set([]))
f11_root.left = f11_root_left_leaf
f11_root_left_leaf.parent=f11_root
f11_root.right = f11_root_right_child
f11_root_right_child.parent=f11_root
f11_root_right_child.left = f11_root_right_child_left_leaf
f11_root_right_child_left_leaf.parent=f11_root_right_child
f11_root_right_child.right = f11_root_right_child_right_leaf
f11_root_right_child_right_leaf.parent=f11_root_right_child
f11 = TreeRule(rtype='lf', root=f11_root,size=tree_size, max_node_id=cur_number, is_good=True)

# f11 = keyword_labelling_func_builder(['love', 'perfect', 'loved', 'nice', 'excellent', 'works', 'loves', 'awesome', 'easy', 'stars', 'soft',
# 	'shoes', 'bought', 'use', 'purchase', 'purchased', 'colors', 'install', 'clean', 'design', 'pair', 'screen', 'comfortable', 'products'], HAM)

f12  = keyword_labelling_func_builder(['returned','broke','battery','cable','fits','install','sturdy','ordered','usb','replacement','brand','installed','unit',
    'batteries','box','warranty','defective','cheaply','durable','advertised'], SPAM)
f13 = keyword_labelling_func_builder(['cute','shirt'], HAM)
f14 = keyword_labelling_func_builder(['fabric','return','money','poor','garbage','poorly','terrible','useless','horrible','returning','flimsy'], SPAM)
f15 = keyword_labelling_func_builder(['pants','looks','toy','color','camera','water','phone','bag','worked','arrived','lasted'], SPAM)

amazon_rules = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15]
return amazon_rules